In [5]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import mendeleev

In [3]:
df = pd.read_csv('CODids.csv',encoding = 'unicode_escape')
df2= pd.read_csv('bandgaps.csv',encoding = 'unicode_escape')
df['bandgaps']=df2['bgs']
df=df.head(1000)

In [4]:

featurelist= ['_chemical_formula_sum', "_chemical_formula_weight",  "_space_group_IT_number", 
"_symmetry_cell_setting",  "_symmetry_space_group_name_H-M",  "_cell_angle_alpha",  "_cell_angle_beta", 
"_cell_angle_gamma",  "_cell_formula_units_Z",  "_cell_length_a",  "_cell_length_b",  "_cell_length_c", 
"_cell_measurement_reflns_used",  "_cell_measurement_temperature",  "_cell_measurement_theta_max",  
"_cell_measurement_theta_min",  "_cell_volume",  "_diffrn_radiation_wavelength",  
"_diffrn_reflns_av_R_equivalents",  "_diffrn_reflns_av_sigmaI/netI",  "_diffrn_reflns_limit_h_max",  
"_diffrn_reflns_limit_h_min",  "_diffrn_reflns_limit_k_max",  "_diffrn_reflns_limit_k_min",  
"_diffrn_reflns_limit_l_max",  "_diffrn_reflns_limit_l_min",  "_diffrn_reflns_number",  
"_diffrn_reflns_theta_full",  "_diffrn_reflns_theta_max",  "_diffrn_reflns_theta_min",  
"_exptl_absorpt_coefficient_mu",  "_exptl_absorpt_correction_T_max",  "_exptl_absorpt_correction_T_min"]

  
# choosing features 
#featurelist = ['_chemical_formula_sum','_chemical_formula_weight',"_symmetry_space_group_name_H-M"]
for x in featurelist:
    stringlist=['_chemical_formula_sum', "_symmetry_space_group_name_H-M"]
    if x in stringlist:
        df[x]= ''
    else:
        df[x]=-100.1

In [5]:
# get info 
for x in df['ids']:
    id= str(x)
    index=df.loc[df['ids'] == x].index[0]
    url=requests.get('http://www.crystallography.net/cod/'+id+'.cif')
    htmltext = url.text
    for y in featurelist:
        matched_lines = [line for line in htmltext.split('\n') if y in line]
        if len(matched_lines)>0:
            if y=="_chemical_formula_sum" or y=="_symmetry_space_group_name_H-M":
                matched_lines= matched_lines[0].split("'")
                df.set_value(index, y, matched_lines[1])
            else:
                matched_lines= matched_lines[0].split(" ")
                k=matched_lines[-1]
                if '(' in k:
                    k= k.split('(')[0]
                if y!="_symmetry_cell_setting":
                    try:
                        k=float(k)
                    except ValueError:
                        k=-100
                if y=="_symmetry_cell_setting":
                    if k== "monoclinic":
                        k=1.0
                    elif k=="triclinic":
                        k=3.0
                    elif k=="orthorhombic":
                        k=5.0
                    else:
                        k=7.0
                df.set_value(index, y, k)


/Users/michellevoong/anaconda2/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:12: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  if sys.path[0] == '':
/Users/michellevoong/anaconda2/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:32: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


KeyboardInterrupt: 

In [1]:
# encoding space group
space_vals = df["_symmetry_space_group_name_H-M"]
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(space_vals)
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
space_vals_encoded = onehot_encoder.fit_transform(integer_encoded)
#print(onehot_encoded[0])
#print(len(onehot_encoded[0]))
space_vals

NameError: name 'df' is not defined

In [ ]:
# encoding symmetry cell setting 
cell_setting = df["_symmetry_cell_setting"]
print(set(cell_setting))
cell_label = LabelEncoder()
cell_encoded = cell_label.fit_transform(cell_setting)
cell_onehot = OneHotEncoder(sparse=False)
cell_encoded = cell_encoded.reshape(len(cell_encoded), 1)
cell_vals_encoded = onehot_encoder.fit_transform(cell_encoded)
#print(cell_vals_encoded[0])
#print(len(cell_vals_encoded[0]))

In [ ]:
X_tsne = TSNE(learning_rate=100).fit_transform(df[df.columns.difference(['bandgaps','_chemical_formula_sum', "_symmetry_space_group_name_H-M"])])
X_pca = PCA().fit_transform(df[df.columns.difference(['bandgaps','_chemical_formula_sum', "_symmetry_space_group_name_H-M"])])

plt.figure(figsize=(10, 5))
plt.subplot(121)
plt.scatter(X_tsne[:, 0], X_tsne[:, 1], c=df['bandgaps'], cmap='RdBu')
plt.subplot(122)
plt.scatter(X_pca[:, 0], X_pca[:, 1], c=df['bandgaps'], cmap='RdBu')
plt.show()

In [ ]:
tsne = TSNE(learning_rate=100).fit_transform(df[df.columns.difference(['ids','_chemical_formula_sum', "_symmetry_space_group_name_H-M"])])
pca = PCA().fit_transform(df[df.columns.difference(['_chemical_formula_sum', "_symmetry_space_group_name_H-M", "ids"])])

plt.figure(figsize=(10, 5))
plt.subplot(121)
plt.scatter(tsne[:, 0], tsne[:, 1], c=df['bandgaps'], cmap='RdBu') # 'RdBu'
plt.subplot(122)
plt.scatter(pca[:, 0], pca[:, 1], c=df['bandgaps'], cmap='RdBu')
plt.show()
# band gap range 0 - 9  
# red-ish values are lower band gaps 
# blue-ish values are higher band gaps 

In [ ]:
# plot chemical formula weight vs. bandgaps 
plt.figure(figsize=(10, 5))
plt.scatter(df['_chemical_formula_weight'],df['bandgaps'],c=df['bandgaps'], cmap='RdBu')
plt.xlim(0, 1500)
plt.ylabel('bandgaps')
plt.xlabel('chemical_formula_weight')
plt.show()

In [ ]:
# plot symmetry space group name vs. bandgaps 
plt.figure(figsize=(10, 5))
plt.scatter(df['_symmetry_space_group_name_H-M'],df['bandgaps'],c=df['bandgaps'], cmap='RdBu')
plt.ylabel('bandgaps')
plt.xlabel('_symmetry_space_group_name_H-M')
plt.xticks(rotation=90)
plt.show()

In [ ]:
def corr_calc(x, y):
    from scipy.stats import pearsonr, spearmanr 
    covariance = np.cov(x,y)
    pcorr, _ = pearsonr(x,y)
    scorr, _ = spearmanr(x,y)
    print('Covariance: ')
    print(covariance)
    print('Pearsons Correlation: ', pcorr)
    print('Spearmans Correlation: ', scorr)

corr_calc(df['_chemical_formula_weight'],df['bandgaps'])

Coulumb Matrix

In [6]:
#load xyz 
xyz_df = pd.read_excel('xyz.xlsx', encoding = 'unicode_escape')

In [7]:
molecules = []
mi = []
for index, row in xyz_df.itertuples(): 
    try:
        if 'Lattice' in row:
            pass
        else:
            line = row.split()
            mi.append(line)
    except TypeError:
        molecules.append(mi)
        mi = []
        pass 

In [8]:
# first 1000 molecules xyz 
molec = molecules[0:1000]
size_molecs = [len(m) for m in molec]
max_molec = max(size_molecs)


In [12]:
def get_coulombmat(molecule): 
    import mendeleev
    """
    takes a molecule as input, each row has molecule name and xyz coordinates 
    returns coulomb matrix
    """
    atoms = []
    xyzmatrix = []
    num_atoms = len(molecule)
    #print(num_atoms)
    for line in molecule:
        elem = mendeleev.element(line[0]) 
        atoms.append(elem)
        cij = np.zeros((num_atoms, num_atoms))
        xyzmatrix.append([[float(line[1]), float(line[2]), float(line[3])]])
    #xyzmatrix = [[atom.position.x, atom.position.y, atom.position.z] for atom in molecule.atoms]
    
    chargearray = [a.atomic_number for a in atoms]
    #print(chargearray)
    for i in range(num_atoms):
        for j in range(num_atoms):
            if i == j:
                cij[i][j] = 0.5 * chargearray[i] ** 2.4  # Diagonal term described by Potential energy of isolated atom
            else:
                dist = np.linalg.norm(np.array(xyzmatrix[i]) - np.array(xyzmatrix[j]))
                cij[i][j] = chargearray[i] * chargearray[j] / dist  # Pair-wise repulsion
    return cij 
    

In [13]:
def pad_cmat(cmat, ref_shape):
    result = np.zeros((ref_shape, ref_shape))
    result[:cmat.shape[0],:cmat.shape[1]] = cmat
    return result
    

In [14]:
molec = molecules[0:100]
c = []
for m in molec:
    padded_c = pad_cmat(get_coulombmat(m), max_molec)
    c.append(padded_c)

In [ ]:
#padded = pad_cmat(c, max_molec)

In [ ]:
# sorted eigenvalues of coulomb matrix 
#sorted_eig = sorted(np.linalg.eigvals(padded), reverse=True)

In [15]:
sorted_eigs = [sorted(np.linalg.eigvals(padded), reverse=True) for padded in c]

In [24]:
len(sorted_eigs)

100

In [16]:
# import band gaps 
bandgaps_df = pd.read_csv('bandgaps.csv', encoding = 'unicode_escape')
bg = bandgaps_df[0:100].to_numpy()
#bg.to_numpy()

Linear Regression - using just eigenspectrum representation of Coulomb Matrix

In [19]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

# test split 
X_train, X_test, y_train, y_test = train_test_split(sorted_eigs, bg, test_size=0.30)#, random_state=42)
reg = LinearRegression().fit(X_train, y_train)
y_pred = reg.predict(X_test)
y_df = pd.DataFrame({'Actual': y_test.flatten(), 'Predicted': y_pred.flatten()})


In [29]:
from sklearn.cluster import SpectralClustering

c_mat = [get_coulombmat(m) for m in molec[0:10]]


list

In [43]:
c_mat = np.empty([10,10])
for m in molec[0:10]:
    c_mat = np.append(c_mat, get_coulombmat(m))

In [44]:
c_mat

array([-1.        ,  0.11111111,  0.11111111, ..., 14.21707579,
       12.45482589, 73.51669472])

In [42]:
clustering = SpectralClustering(n_clusters=2,assign_labels="discretize",random_state=0)
clustering.fit(c_mat)

/Users/michellevoong/anaconda2/envs/py37/lib/python3.7/site-packages/sklearn/cluster/spectral.py:462: UserWarning: The spectral clustering API has changed. ``fit``now constructs an affinity matrix from data. To use a custom affinity matrix, set ``affinity=precomputed``.
  warnings.warn("The spectral clustering API has changed. ``fit``"


SpectralClustering(affinity='rbf', assign_labels='discretize', coef0=1,
                   degree=3, eigen_solver=None, eigen_tol=0.0, gamma=1.0,
                   kernel_params=None, n_clusters=2, n_init=10, n_jobs=None,
                   n_neighbors=10, random_state=0)

In [33]:
X = np.array([[1, 1], [2, 1], [1, 0],[4, 7], [3, 5], [3, 6]])
clustering = SpectralClustering(n_clusters=2,assign_labels="discretize",random_state=0).fit(X)
X

array([[1, 1],
       [2, 1],
       [1, 0],
       [4, 7],
       [3, 5],
       [3, 6]])